In [2]:
import pandas as pd
import requests
import json
from Resources.config import app_token

In [7]:
# Crimes - 2001 to present
# https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2
url_crime_data = "https://data.cityofchicago.org/resource/ijzp-q8t2.json"
date_filter="$WHERE=DATE BETWEEN '2014-01-01' AND '2019-01-01'"
crime_records_limit = 1000
#1345713
url_crime_data_with_filter = f"{url_crime_data}?{date_filter}&$limit={crime_records_limit}&$$app_token={app_token}"
crime_data_json = requests.get(url_crime_data_with_filter).json()
#len(crime_data_json)
#print(json.dumps(crime_data_json, indent=4, sort_keys=True))

In [8]:
# Convert to pandas DataFrame
crime_data_df = pd.DataFrame.from_records(crime_data_json)
#drop extra columns
crime_data_df = crime_data_df.drop(['x_coordinate','y_coordinate','latitude','longitude','location'],axis=1)
#crime_data_df.head(2000)

In [9]:
# Boundaries - Community Areas (current)
# https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6

url_comm_area = "https://data.cityofchicago.org/resource/igwz-8jzy.json"
field_filter = "$select=community,area_numbe"
url_comm_area_with_filter = f"{url_comm_area}?{field_filter}&$order=area_numbe&$$app_token={app_token}"
comm_data_json = requests.get(url_comm_area_with_filter).json()

In [10]:
# Convert to pandas DataFrame
comm_data_df = pd.DataFrame.from_records(comm_data_json)
comm_data_df['area_numbe'] = pd.to_numeric(comm_data_df['area_numbe'],downcast='signed')
comm_data_df_sorted = comm_data_df.head(2000).sort_values(by='area_numbe').reset_index(drop=True)
#comm_data_df_sorted

In [11]:
# 311 Service Requests - Street Lights - All Out
# https://data.cityofchicago.org/Service-Requests/311-Service-Requests-Street-Lights-All-Out-No-Dupl/756k-itxx
url_service_requests = "https://data.cityofchicago.org/resource/zuxi-7xem.json"
status_filter = "$select=creation_date,status,completion_date,service_request_number,type_of_service_request,community_area&$where=creation_date between '2014-01-01' AND '2019-01-01'"
service_requests_limit = 100
url_service_requests_with_filter = f"{url_service_requests}?{status_filter}&$limit={service_requests_limit}&$$app_token={app_token}"
service_requests_json = requests.get(url_service_requests_with_filter).json()

In [12]:
# Convert to pandas DataFrame
service_requests_df = pd.DataFrame.from_records(service_requests_json)
#drop extra columns
service_requests_df.head(2000)

,creation_date,status,completion_date,service_request_number,type_of_service_request,community_area
0,2014-01-01T00:00:00.000,Completed,2014-01-06T00:00:00.000,14-00002286,Street Lights - All/Out,29
1,2014-01-01T00:00:00.000,Completed - Dup,2014-01-06T00:00:00.000,14-00001878,Street Lights - All/Out,44
2,2014-01-01T00:00:00.000,Completed,2014-01-03T00:00:00.000,14-00001931,Street Lights - All/Out,30
3,2014-01-01T00:00:00.000,Completed,2014-01-03T00:00:00.000,14-00001932,Street Lights - All/Out,30
4,2014-01-01T00:00:00.000,Completed - Dup,2014-01-08T00:00:00.000,14-00001100,Street Lights - All/Out,68
...,...,...,...,...,...,...
95,2014-01-02T00:00:00.000,Completed,2014-01-07T00:00:00.000,14-00002657,Street Lights - All/Out,3
96,2014-01-02T00:00:00.000,Completed,2014-01-03T00:00:00.000,14-00008044,Street Lights - All/Out,25
97,2014-01-02T00:00:00.000,Completed,2014-01-03T00:00:00.000,14-00007607,Street Lights - All/Out,73
98,2014-01-02T00:00:00.000,Completed - Dup,2014-01-08T00:00:00.000,14-00007358,Street Lights - All/Out,43
